### ถูกต้อง

In [ ]:
import requests
import os
import json
import pandas as pd

url = "https://power.larc.nasa.gov/api/temporal/daily/point"
parameters = ['GWETPROF', 'ALLSKY_SFC_SW_DWN', 'ALLSKY_SFC_LW_DWN', 'PS', 'PW', 'RH2M']
target = '../Satun-Phang-Nga/forcasting/'

# Process each file in the target directory
for i, file in enumerate(os.listdir(target)):
    file_path = os.path.join(target, file)
    df = pd.read_excel(file_path)
    
    # Print columns to verify names
    print(f"Columns in file {file}: {df.columns.tolist()}")
    
    # Ensure that 'date', 'latitude', and 'longitude' columns are present
    if 'date' not in df.columns or 'latitude' not in df.columns or 'longitude' not in df.columns:
        print(f"Missing one of the required columns in file {file}")
        continue
    
    df['date'] = pd.to_datetime(df['date'])
    
    start = df['date'].min().strftime('%Y%m%d')
    end = df['date'].max().strftime('%Y%m%d')
    latitude = df['latitude'].iloc[0]
    longitude = df['longitude'].iloc[0]
    community = 'AG'
    
    params = {
        "parameters": ','.join(parameters),
        "start": start,
        "end": end,
        "latitude": latitude,
        "longitude": longitude,
        "community": community
    }

    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        if 'properties' in data:
            # Extract the parameter data
            parameters_data = data['properties']['parameter']
            
            # Create an empty DataFrame to hold all parameters
            df_weather = pd.DataFrame()
            
            # Iterate through each parameter and add it to the DataFrame
            for param in parameters:
                if param in parameters_data:
                    param_data = parameters_data[param]
                    df_param = pd.DataFrame(list(param_data.items()), columns=['date', param])
                    df_param['date'] = pd.to_datetime(df_param['date'], format='%Y%m%d')
                    
                    # Merge with existing DataFrame
                    if df_weather.empty:
                        df_weather = df_param
                    else:
                        df_weather = pd.merge(df_weather, df_param, on='date', how='outer')
                        
            # Define output file path
            output_file = os.path.join(target, f"weather_data_{i}.xlsx")
            # Save DataFrame to Excel
            df_weather.to_excel(output_file, index=False)
            print(f"Saved data for file {file} to {output_file}")
        else:
            print(f"No 'properties' in data for file {file}")
    else:
        # Print detailed error information
        print(f"Error: {response.status_code} for file {file}")
        print("Response content:", response.text)
